In [42]:
import csv
import random
import numpy as np
import matplotlib as mpl
import scipy.stats as sps

survey_files=["./survey_fcasts.yr4.csv"]
#, "./survey_fcasts.yr2.csv", "./survey_fcasts.yr3.csv", "./survey_fcasts.yr4.csv"]

preds=[]

year=1

for n in survey_files:
    f=open(n)
    predreader=csv.DictReader(f)
    for entry in predreader:
        entry['year']=year
        entry['cond']=int(entry['cond'])
        entry['value']=float(entry['value'])
        preds.append(entry)
    year=year+1

questions=[]

qfile=open("ifps.csv")
qreader=csv.DictReader(qfile)

for entry in qreader:
    questions.append(entry)

qresults=dict()

for q in questions:
    # this shouldn't happen, the questions should be unique
    if qresults.__contains__(q['ifp_id']):
        print("duplicate question ", q['ifp_id'])
    qresults[q['ifp_id']]=q['outcome']

In [43]:
for p in preds:
    p['outcome']=qresults[p['ifp_id']]

In [44]:
user_forecasts=dict()
for p in preds:
    uid=p['user_id']
    if not user_forecasts.__contains__(uid):
        user_forecasts[uid]=dict()
        user_forecasts[uid]['forecasts']=np.array([])
        user_forecasts[uid]['options']=np.array([])
        user_forecasts[uid]['outcomes']=np.array([])
    user_forecasts[uid]['forecasts']=np.append(user_forecasts[uid]['forecasts'], p['value'])
    user_forecasts[uid]['options']=np.append(user_forecasts[uid]['options'], p['answer_option'])
    user_forecasts[uid]['outcomes']=np.append(user_forecasts[uid]['outcomes'], p['outcome'])

# In the file survey_fcasts.yr1.csv there are a couple of forecasts made by the user "NULL"
# I don't know what this means: I suspected at first that those were forecasts on "voided" questions,
# but it turns out that that's not the case.
# To be sure, I'll delete the forecasts by that user from the data.

#user_forecasts.pop('NULL')
users=list(user_forecasts.keys())

def individual_score(user):
    return np.mean((user['forecasts']-(user['options']==user['outcomes']))**2)

individual_scores=np.array([])

for k in user_forecasts.keys():
    u=user_forecasts[k]
    u['brier']=individual_score(u)
    individual_scores=np.append(individual_scores, u['brier'])

individual_scores=np.sort(individual_scores)

In [45]:
user_teams=dict()
for p in preds:
    uid=p['user_id']
    if not user_teams.__contains__(uid):
        user_teams[uid]=dict()
        user_teams[uid]['teams']=set()
    user_teams[uid]['teams']
    if not p['team'] in user_teams[uid]['teams']:
        user_teams[uid]['teams'].add(p['team'])

for k in user_teams:
    if len(user_teams[k]['teams'])!=1:
        print(k, user_teams[k])

In [41]:
GROUP_SCORE_SAMPLES=1000
MAX_GROUP_SIZE=50

group_scores=dict()

def group_score(user_forecasts, users):
    forecasts=np.array([])
    results=np.array([])
    for u in users:
        udata=user_forecasts[u]
        forecasts=np.append(forecasts, udata['forecasts'])
        results=np.append(results, udata['options']==udata['outcomes'])
    return np.mean((forecasts-results)**2)

for size in range(2, MAX_GROUP_SIZE+1):
    samples=[]
    for n in range(0, GROUP_SCORE_SAMPLES):
        sample_group=random.sample(users, size)
        score=group_score(user_forecasts, sample_group)
        samples.append(score)
    group_scores[size]=np.mean(np.array(samples))

print(group_scores)

for k in group_scores.keys():
    print(k, len((np.where(individual_scores<group_scores[k]))[0])/len(individual_scores))

{2: 0.17936101850298253, 3: 0.18043972278822742, 4: 0.1789916570720153, 5: 0.1783850677829238, 6: 0.17812073502925768, 7: 0.1778820459076874, 8: 0.1764100172591956, 9: 0.17702866366026485, 10: 0.17802154295148634, 11: 0.1770885881994059, 12: 0.175798923970029, 13: 0.17639933507615763, 14: 0.17615937255835637, 15: 0.175694967475053, 16: 0.17549631911117955, 17: 0.17527014683907685, 18: 0.1767434539686643, 19: 0.1754158963624104, 20: 0.17453543573095048, 21: 0.1752063728680809, 22: 0.17499888345765982, 23: 0.17565776197121574, 24: 0.17454435986751798, 25: 0.17469873450023482, 26: 0.174623442111147, 27: 0.175548184094046, 28: 0.17510504046037972, 29: 0.17471173855935052, 30: 0.1747568770642281, 31: 0.1752857743719339, 32: 0.1746037624267343, 33: 0.17514032135432753, 34: 0.1746504252377327, 35: 0.174838343767607, 36: 0.17401667680415583, 37: 0.1751274916638128, 38: 0.17449841646921158, 39: 0.17481683889376456, 40: 0.17435886328285705, 41: 0.17457665144042483, 42: 0.17410879387842954, 43: 0